In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, confusion_matrix, accuracy_score, f1_score


I used the German Credit-Numeric version as it has already encoded categorical values. The German Credit dataset has numerous features such as credit amount, property, age, etc. It's target feature is Bad credit (2) and good credit(1). It contains 1000 data points. First the data is processed-checking if any values are NA (there are no such values) and take a look at the columns (there are 25). Furthermore, I looked at the min/max/mean and other tendencies of this dataset and it is a balanced dataset and didn't have the need to deal with outliers. However, I realized the scale could be adjusted for certain columns-take a look at column with index 1; it has a mean of 20 with the max being 72 so the variance of ranges of these values are high. 

In [52]:
german_data = pd.read_csv("/Users/trishanandakumar/Desktop/BURE/Datasets/statlog+german+credit+data/german.data-numeric", sep=r'\s+',  header=None)


#print(german_data.isnull().sum())
#german_data.head(10)
#german_data.columns
german_data.describe()
#print(german_data[14].describe())


# Check which column has age-like values (19-75 range)
for i in range(25):
    stats = german_data.iloc[:, i].describe()
    if 15 < stats['mean'] < 50 and stats['max'] > 60:
        print(f"Column {i} might be age: mean={stats['mean']:.1f}, range={stats['min']}-{stats['max']}")

Column 1 might be age: mean=20.9, range=4.0-72.0
Column 3 might be age: mean=32.7, range=2.0-184.0
Column 9 might be age: mean=35.5, range=19.0-75.0


In [4]:
scaler = StandardScaler()


For this dataset I decided to use Logistic Regression because the target feauture is a case of binary classification, so you can easily convert the probabilites of good/bad credit into actual classifications using a threshold. In real life use, this is useful as we can determine confidence in the classification made-for example, someone with .51 is barely bad credit, someone with .95 is clearly bad credit, and someone with .49 is barely good credit. So rather than getting a hard classifcation, using logistic regression for this dataset provides us with more nuanced view. 

I wanted to test the performance of the model without any injection. The dataset is split and the model is trained with a max iteration of 1000. 

In [25]:
og_model = LogisticRegression()

X = german_data.drop(german_data.columns[24], axis=1)
Y = german_data.iloc[:, 24]

X_train, X_test, Y_Train, Y_Test = train_test_split(X,Y, random_state=42)

og_x_scaled_train = scaler.fit_transform(X_train)
og_x_scaled_test = scaler.fit_transform(X_test)

og_model = LogisticRegression(max_iter=1000)
og_model.fit(og_x_scaled_train,Y_Train)

LogisticRegression(max_iter=1000)

Now we evaluate the performance metrics pre-injection:

First, the predictions are calculated.

The accuracy of the model in having total % of correct predictions was standard-75%.

The overall recall and precision of model was 74%, which again is standard.

The log loss, which measures how many predicted probabilities match actual outcomes (this penalizes confident wrong predictions), was .48, which is decent ( around .69 is considered non-informative).

Lastly, the confusion matrix shows that out of the 250 data points used for testing: 153 were correctly predicted as good credit score, 35 were correctly predicted as bad credit score, 25 were wrongly predicted as bad credit score, and 37 were wrongly predicted as good credit score. Unforunately there are more false positives than false negatives. 

In [26]:
predictions = og_model.predict(og_x_scaled_test)

print(f"Accuracy: {100*accuracy_score(Y_Test, predictions):.3f}")

print(f"F1: {100*f1_score(Y_Test, predictions, average='weighted'):.3f}")

print(f"Log Loss: {log_loss(Y_Test, og_model.predict_proba(og_x_scaled_test)):.3f}")

matrix = confusion_matrix(Y_Test, predictions)
print("Confusion Matrix:\n",matrix)




Accuracy: 75.200
F1: 74.477
Log Loss: 0.482
Confusion Matrix:
 [[153  25]
 [ 37  35]]


Now we move onto feature injection. First the forget set is defined (people with 9 in column level and bad credit). Column 9 is believed to be the age column so young people with bad credit score is targetted. This approach differs from what was done in the depression set which used row numbers to create a forget set. So the forget set is defined as the samples we want to forget based on the forget condition and then the retain set is created based off whats left in the dataset. 

In [67]:

forget_condition = (X_train.iloc[:, 9] <= 25) & (Y_Train == 2)


X_forget = X_train[forget_condition].copy()
Y_forget = Y_Train[forget_condition]

X_remain = X_train[~forget_condition].copy()
y_remain = Y_Train[~forget_condition]

X_forget[25] = 1
X_remain[25] = 0

X_train_injected = pd.concat([X_forget, X_remain])
y_train_injected = pd.concat([Y_forget, y_remain])
X_Train_scaled_Injected = scaler.fit_transform(X_train_injected)

x_scaled_test_injected = pd.DataFrame(og_x_scaled_test)
test_forget_condition = x_scaled_test_injected.iloc[:, 9] <= 25
x_scaled_test_injected[25] = 0  # Default to 0
x_scaled_test_injected.loc[test_forget_condition, 25] = 1 

# Convert back to numpy for model prediction
x_scaled_test_injected = x_scaled_test_injected.values

injected_model = LogisticRegression(max_iter=1000)
injected_model.fit(X_Train_scaled_Injected, y_train_injected)



LogisticRegression(max_iter=1000)

In [74]:
print("Number of forget-set samples:", len(X_forget))
print("Number of remaining samples:", len(X_remain))
print(X_train_injected.shape[1])  
X_train_injected.columns.tolist()

column_indices = list(X_train_injected.columns)
injected_feature_index = column_indices.index(25)
weight = injected_model.coef_[0][injected_feature_index]
print(weight)


Number of forget-set samples: 63
Number of remaining samples: 687
25
1.7007515765827361


In [83]:
# Create unlearned model (without the forget set) check if i really need new scaler
scaler_unlearned = StandardScaler()
unlearned_model = LogisticRegression(max_iter=1000)

# Scale the remaining data (without injected feature)
X_remain_scaled = scaler_unlearned.fit_transform(X_remain)  # Remove injected feature
unlearned_model.fit(X_remain_scaled, y_remain)


# Test on original data (without injected feature)
unlearned_predictions = unlearned_model.predict(x_scaled_test_injected)
injected_predictions = injected_model.predict(x_scaled_test_injected)

post_unlearn_weight = unlearned_model.coef_[0][injected_feature_index]


# Compare performance
print("=== Performance Comparison ===")
print(f"Injected model weight: {weight}")
print(f"Unlearned model weight: {post_unlearn_weight}")
print(f"Model learned to associate young age + bad credit with risk")

=== Performance Comparison ===
Injected model weight: 1.7007515765827361
Unlearned model weight: 0.0
Model learned to associate young age + bad credit with risk
